In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("data/train.xlsx")
df_train = pd.DataFrame()
df_train['Date'] = pd.to_datetime(df['dt'], format="%d.%m.%Y")
df_train['Price'] = df['Цена на арматуру']
df = pd.read_excel("data/test.xlsx")
df_test = pd.DataFrame()
df_test['Date'] = pd.to_datetime(df['dt'], format="%Y.%m.%d")
df_test['Price'] = df['Цена на арматуру']
df = pd.concat((df_train, df_test)).reset_index(drop=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    428 non-null    datetime64[ns]
 1   Price   428 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 6.8 KB


In [4]:
df.describe()

,Date,Price
count,428,428.000000
mean,2019-02-07 12:00:00,39557.920561
min,2015-01-05 00:00:00,21000.000000
25%,2017-01-21 06:00:00,31650.000000
50%,2019-02-07 12:00:00,36150.000000
75%,2021-02-23 18:00:00,43162.500000
max,2023-03-13 00:00:00,79000.000000
std,NaN,13052.612976


Создаются X (признаки) и y (целевая переменная — Price).
Добавляются новые столбцы, такие как Price_target, Price_source и Price_Diff, которые отражают изменения цен.
Удаляются ненужные колонки (Date, Price_target).
Исключается первая строка (X.drop([0])).

In [5]:
X = df.drop(columns=['Price']).iloc[:-1, :]
y = df['Price'].values[1:]
X['Price_target'] = y
X['Price_source'] = df['Price'].values[:-1]
price_diff = X['Price_source'] - X['Price_target']
y = X['Price_target']
X = X.drop(columns=['Price_target', 'Date'])
X = X.drop([0])
X['Price_Diff'] = price_diff.values[:-1]
X

,Price_source,Price_Diff
1,32500,400
2,32000,500
3,30700,1300
4,30300,400
5,30300,0
...,...,...
422,44500,-1700
423,44500,0
424,46000,-1500
425,49200,-3200


In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor, early_stopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Пересинхронизация индексов
y = y[X.index]

# Разделение данных на обучающую и тестовую выборки
X_train, X_holdout, y_train, y_holdout = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=42
)

# Инициализация модели
model0 = LGBMRegressor(n_estimators=3000, max_depth=7, learning_rate=0.03,
                       objective='mae', metric='mae',
                       random_state=42, verbose=100)

# Разделение данных на обучающую и валидационную выборки
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Обучение модели с использованием early_stopping
model0.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
           callbacks=[early_stopping(stopping_rounds=100)])

# Предсказания
y_pred = model0.predict(X_valid)

# Вычисление метрик
mae = mean_absolute_error(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
r2 = r2_score(y_valid, y_pred)

# Вывод результатов
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")


[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.093750
[LightGBM] [Debug] init for col-wise cost 0.000001 seconds, init for row-wise cost 0.000368 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 113
[LightGBM] [Info] Number of data points in the train set: 272, number of used features: 2
[LightGBM] [Info] Start training from score 36850.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 4 and depth = 2
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 4 and depth = 2
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 4 and depth = 2
[L

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Пересинхронизация индексов
y = y[X.index]

# Разделение данных на обучающую и тестовую выборки
X_train, X_holdout, y_train, y_holdout = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=42
)

# Инициализация модели
model0 = RandomForestRegressor(n_estimators=1000, max_depth=7, random_state=42, verbose=1)

# Разделение данных на обучающую и валидационную выборки
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Обучение модели
model0.fit(X_train, y_train)

# Предсказания
y_pred = model0.predict(X_valid)

# Вычисление метрик
mae = mean_absolute_error(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
r2 = r2_score(y_valid, y_pred)

# Вывод результатов
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s


MAE: 1081.4438
RMSE: 1773.2796
R2 Score: 0.9790


[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.0s
